<img src="images/dask_logo.svg" alt="Dask" style="height: 250px;"/> 

# [Dask](https://docs.dask.org/en/latest/)

Dask is a Python library which allows large data processing tasks to be automatically broken up into chunks and executed on a cluster of workers.

Dask is integrated with many Python tools used for data processing such as:
- [Pandas](https://examples.dask.org/dataframe.html) -> DaskDataFrame
- [Numpy](https://examples.dask.org/array.html) -> DaskArray
- [xarray](https://examples.dask.org/xarray.html)
- [Iris](https://scitools.org.uk/iris/docs/latest/userguide/real_and_lazy_data.html)
- [etc](https://dask.org/)...

# [Distributed](https://distributed.dask.org/en/latest/)

This is another Python library which works with Dask to interface with and execute processing tasks on cluster-like computer resource e.g. HPC, analysis cluster, cloud computing. It does this by helping dask interface with orchestration software e.g. jobqueue scheculers like [SLURM](https://github.com/dask/dask-jobqueue), [Kubernetes](https://kubernetes.dask.org/en/latest/), [Tensorflow](https://github.com/dask/dask-tensorflow).


----

## Starting a cluster and accessing it through a client

First we need to import the right libraries

In [ ]:
import os
import distributed
import dask
from dask_kubernetes import KubeCluster
from dask import array as da

## Start Cluster

Here we are going to create a Kubernetes cluster on our cloud based platform (using `KubeCluster`). Kubernetes will automatically request and set up the resources needed.<br>
The cluster will be adaptive, which means that it will scale up the number of workers when needed and scale down when fewer workers are required.

In [ ]:
cluster = KubeCluster()
cluster.adapt()
cluster

Clicking on the **`Dashboard`** url will open a dashboard in a new browser tab. Here you can monitor the state of your dask workers and progress of an calculations.

![Dask dashboard](images/dask_dashboard.png)

## Connect a distributed client

Not all types of clusters return a useful information widget like a `KubeCluster`. For those we can pass the cluster object to a `distributed.Client`, which will render us useful things like number of workers in the cluster, but also tools to control how many workers are in the cluster.

In [ ]:
client = distributed.Client(cluster)
client

## Testing our cluster with a random array

To test that our cluster works (and to see it in action) we can create a `DaskArray` of random numbers and calculate its mean.

The following code creates a **2500x2500x2500** array of random numbers, represented as many numpy arrays of size **500x500** (or smaller if the array cannot be divided evenly).<br>
In this case there are **125 (5x5x5)** numpy arrays of size **500x500**.

In [ ]:
import dask.array as da
x = da.random.random((2500, 2500, 2500), chunks=(500, 500, 500))
x

Use standard NumPy syntax to calculate the mean of the array `x`.

In [ ]:
z = x.mean()
z

Call `.compute()` when you want to start the calculation.

You may want to watch the status page during computation.

In [ ]:
z.compute()

This demonstrates 'lazy' nature of this calculation - we stated all the computations we wanted to perform (`define array -> calculate mean`) but nothing is calculated until we ask dask to do so (`.compute`) or dask has to (e.g. because we have generated a plot of the data).